In [8]:
# Módulos
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [9]:
# Dados
Moveis = [0, 'Escrivaninha 1', 'Escrivaninha 2', 'Mesa 1', 
          'Armário 1', 'Armário 2', 'Armário 3', 'Prateleira 1']
Equipes = [1, 2]
tempo_producao = {(0, 1): 0, (0, 2): 0,
                  ('Escrivaninha 1', 1): 3, ('Escrivaninha 1', 2): 6,
                  ('Escrivaninha 2', 1): 3, ('Escrivaninha 2', 2): 6,
                  ('Mesa 1', 1): 1, ('Mesa 1', 2): 2, 
                  ('Armário 1', 1): 2, ('Armário 1', 2): 4, 
                  ('Armário 2', 1): 2, ('Armário 2', 2): 4, 
                  ('Armário 3', 1): 2, ('Armário 3', 2): 4, 
                  ('Prateleira 1', 1): 1, ('Prateleira 1', 2): 2}

In [10]:
# Modelo
model = ConcreteModel()

# Conjuntos
model.M = Set(initialize=Moveis)
model.E = Set(initialize=Equipes)
M, E = model.M, model.E

# Parâmetros
model.p = Param(M * E, initialize=tempo_producao, within=NonNegativeReals)
model.I = Param(initialize=inicio, within=NonNegativeReals)
p, I = model.p, model.I

# Variáveis de decisão
model.x = Var(M * E, within=Binary)
model.C = Var(E, within=NonNegativeReals)
model.C_max = Var(within=NonNegativeReals)
x, C, C_max = model.x, model.C, model.C_max

# Função objetivo
model.obj = Objective(sense=minimize,
                      expr = C_max)

# Restrições
# Definição do instante final de processamento
model.r_proc_final = ConstraintList()
for k in E:
    model.r_proc_final.add(expr = C[k] == sum(p[i, k] * x[i, k] for i in M))
# Definição do makespan
model.r_mksp = ConstraintList()
for k in E:
    model.r_mksp.add(expr = C_max >= C[k])
# Todas as tarefas são processadas
model.r_proc = ConstraintList()
for i in M:
    model.r_proc.add(expr = sum(x[i, k] for k in E) == 1)

# model.pprint()

In [11]:
# Resolução
solver = SolverFactory('cplex')
resultado = solver.solve(model, tee=False)

In [12]:
# Apresentação de resultados
if (resultado.solver.status == SolverStatus.ok) and (resultado.solver.termination_condition != TerminationCondition.infeasible):
    for i in M:
        for k in E:
            if value(x[i, k]) == 1 and i != 0:
                print('O móvel {} é processado pela equipe {}'.format(i, k))
    print()
    print('O makespan mínimo é de {:.0f} horas.'.format(value(model.obj)))
else:
    print(resultado.solver.termination_condition)

O móvel Escrivaninha 1 é processado pela equipe 2
O móvel Escrivaninha 2 é processado pela equipe 1
O móvel Mesa 1 é processado pela equipe 1
O móvel Armário 1 é processado pela equipe 1
O móvel Armário 2 é processado pela equipe 2
O móvel Armário 3 é processado pela equipe 1
O móvel Prateleira 1 é processado pela equipe 1

O makespan mínimo é de 10 horas.
